# 1. Prefer Exceptions to returning None 

Functions that return None to indicate special meaning are error prone because none and other values (e.g. zero, the empty string) all evaluate to false in conditional expressions.
In the below example, result_2 is perfectly valid and returns zero, but the handling is not proper.

In [2]:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None
   
result_1 = divide(5, 0)
if not result_1:
    print("result_1 is not ok due to invalid inputs")
    
result_2 = divide(0, 5)
if not result_2:
    print("result_2 is not ok due to invalid inputs")

result_1 is not ok due to invalid inputs
result_2 is not ok due to invalid inputs


It is clearer to raise exceptions to indicate special situations instead of returning None. Expect the calling code to handle exceptions properly when they are documented.

In [7]:
def divide(a, b):
    """
    params: a: numerator
    params: b: denominator
    returns: the result when b is non-zero and ValueError when b is zero.
    """
    try:
        return a / b
    except ZeroDivisionError:
        return ValueError('Invalid inputs')
    
x, y = 5, 2
try:
    result = divide(x, y)
except ValueError:
    print("Invalid Inputs")
else:
    print(f"Result is {result}")
    

x, y = 5, 0
try:
    result = divide(x, y)
except ValueError:
    print("Invalid Inputs")
else:
    print(f"Result is {result}")

Result is 2.5
Result is Invalid inputs


# 2. What are Closures in Python

We have a closure in Python when a nested function references a value in its enclosing scope.

The criteria that must be met to create closure in Python are summarized in the following points.

- We must have a nested function (function inside a function).
- The nested function must refer to a value defined in the enclosing function.
- The enclosing function must return the nested function.

In [3]:
def make_multiplier_of(n):
    def multiplier(x):
        return x * n
    return multiplier

times3 = make_multiplier_of(3)
times5 = make_multiplier_of(5)

print(times3(10))
print(times5(10))

30
50


All function objects have a __closure__ attribute that returns a tuple of cell objects if it is a closure function. Referring to the example above, we know times3 and times5 are closure functions.

The cell object has the attribute cell_contents which stores the closed value.

In [6]:
times3.__closure__

(<cell at 0x000001F279D30B20: int object at 0x000001F275026970>,)

In [7]:
times5.__closure__[0].cell_contents

5

# 3. Return Generators instead of Lists

The simplest choice for a function that produce a sequence of results is to return a list.
For example, say you want the index of ecery word in a string.

In [3]:
def index_words(text):
    result = []
    if text:
        result.append(0)
    for index, letter in enumerate(text):
        if letter == " ":
            result.append(index + 1)
    return result

In [4]:
text_input = "Jupyter notebooks are a nice way to practice coding..."
result = index_words(text_input)
# printing the first five word indices
print(result[:5])

[0, 8, 18, 22, 24]


A better way to write this function is using a generator. Generators are functions that use the yield expressions.

When called, generator functions do not actually run but instead immediately return an iterator. With each call to the next built-in function, the iterator will advance the generator to its next yield expression. Each value passed to yield by the generator will be returned by the iterator to the caller.

The iterator returned by the generator call can be easily converted back to a list by passing it to the list built-in function.

In [8]:
def index_words_iter(text):
    if text:
        yield 0
    for index, letter in enumerate(text):
        if letter == " ":
            yield index + 1

In [9]:
import itertools

text_input = "Jupyter notebooks are a nice way to practice coding..."
result = index_words_iter(text_input)
# printing the first five word indices
print(list(itertools.islice(result, 0, 5)))

[0, 8, 18, 22, 24]


Another problem with index_words is that it requires all results to be stored in the list before being returned. For huge inputs, this can lead to your program to run out of memory and crash.
In contrast, a generator version of this program can be easily adapted to take inputs of arbitrary length.

In [10]:
def index_file(handle):
    offset = 0
    for line in handle:
        if line:
            yield offset
        for letter in line:
            offset += 1
            if letter == " ":
                yield offset

In [ ]:
with open("filename") as f:
    result = index_file(f)
    print(list(itertools.islice(result, 0, 5)))

The only gotcha of defining generators like this is that the callers must be aware that the iterators returned are stateful and can't be re-used.